```
token space layout
0-17 - codebooks channel 1
18-35 - codebooks channel 2
36 - control codes
37 - genre
38 - artist
39 - length
40 - tempo
41 - mood
42 - instruments
-63 - ?

control lane:
0 - nothing
1-6 - ~0/3/6/12/24/48 seconds before end
```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# dataset prep test

In [ ]:
!pip install descript-audio-codec yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 47.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%cd /content
!git clone https://github.com/RWKV/RWKV-infctx-trainer.git
!git clone https://github.com/Abel2076/json2binidx_tool.git
!pip install lightning==2.0.5 deepspeed==0.10.0 datasets transformers ninja numexpr jsonargparse 'jsonargparse[signatures]' lm-dataformat ftfy sentencepiece tokenizers wandb papermill

/content
Cloning into 'RWKV-infctx-trainer'...
remote: Enumerating objects: 7527, done.
remote: Counting objects: 100% (1075/1075), done.
remote: Compressing objects: 100% (401/401), done.
remote: Total 7527 (delta 743), reused 965 (delta 670), pack-reused 6452
Receiving objects: 100% (7527/7527), 29.75 MiB | 15.84 MiB/s, done.
Resolving deltas: 100% (5156/5156), done.
Cloning into 'json2binidx_tool'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 53 (delta 12), reused 7 (delta 7), pack-reused 36
Receiving objects: 100% (53/53), 1003.34 KiB | 2.01 MiB/s, done.
Resolving deltas: 100% (21/21), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 836.6/836.6 kB 60.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 38.8 MB/s eta 0:00:00
     ━━━━━

In [ ]:
#@title patch data.py

%%writefile /content/RWKV-infctx-trainer/RWKV-v5/src/data.py

from lightning import LightningDataModule

from torch.utils.data import DataLoader

import wandb
from datasets import load_from_disk, load_dataset, Dataset
from transformers import PreTrainedTokenizerFast, AutoTokenizer
from multiprocessing import cpu_count
num_cpus = cpu_count()
num_workers = cpu_count() if cpu_count() < 8 else 8

# Get the script directory
import os
SRC_DIR = os.path.dirname(os.path.realpath(__file__))

# World tokenizer
from .dataflow.trie_tokenizer import world_tokenizer_encode
import numpy as np

# We have to extract out the prepare function to be "outside the class"
# else it will not be hashed / serialized properly, and will produce the following error:
#
# ```
# Parameter 'function'=<function RWKVDataModule.prepare_data.<locals>.map_tokenizer at 0x7f7672c5e340> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
# ```
def prepare_data_static(**kargs):

    # Check if skip_datapath_setup is enabled
    # useful for extra large datasets
    if kargs["skip_datapath_setup"] == True:
        return

    # Special handling of world_add_endoftext_token (if enabled)
    if kargs["world_add_endoftext_token"]:
        world_add_endoftext_token = True
    else:
        world_add_endoftext_token = False

    # Source data processing
    if kargs["source"] is not None:
        if kargs["tokenizer"] is None:
            raise ValueError('Tokenizer must be specified if source is specified')

        # Special handling for binidx
        #--------------------------------

        if kargs["tokenizer"] == "binidx":
            from .dataflow.binidx import MMapIndexedDataset

            # Load the MMapIndexedDataset from the source path
            mmap_dataset = MMapIndexedDataset(kargs["source"])
            mmap_dataset_len = mmap_dataset.__len__()

            # Torch dataset generator wrapper
            def gen():
                for idx in range(mmap_dataset_len):
                    # cast to supported types, note that np.int32 limit is 2,147,483,647
                    # - so unless we have a tokenizer that exceeds this, it should be ok
                    tokens = np.array(mmap_dataset.get(idx), dtype=np.int32)
                    tokens = tokens.reshape(-1, kargs["multitoken_width"])
                    yield {
                        'input_ids': tokens,
                        'token_type_ids': [0] * len(tokens),
                        'attention_mask': [1] * len(tokens)
                    }

            # Load the huggingface dataset from the generator
            src_dataset = Dataset.from_generator(gen)

            # Train/test split
            test_split = kargs["test_split"]
            # The minimum test size is 1, if not we will get errors in the trainer?
            if test_split <= 0 or test_split <= 0.0:
                test_split = 1
            split_dataset = src_dataset.train_test_split(
                test_size=test_split,shuffle=kargs["test_split_shuffle"],
                seed=42 #Fixed seed, to prevent train/test reshuffling between test runs
            )

            # Save the dataset to disk
            split_dataset.save_to_disk(kargs["data_path"])
            # Does nothing else (done)
            return

        # Reverting back to general purpose HF dataset / tokenizer handling
        #--------------------------------

        load_dataset_params = {
            'path': kargs["source"],
            'num_proc': num_cpus
        }

        # Handle advance params (if set)
        if kargs["source_data_dir"] is not None:
            load_dataset_params['data_dir'] = kargs["source_data_dir"]
        if kargs["source_dataset_params"] is not None:
            source_dataset_params = kargs["source_dataset_params"]
            for k, v in source_dataset_params.items():
                load_dataset_params[k] = v

        # Load the dataset
        src_dataset = load_dataset(**load_dataset_params)

        # If for some reason the dataset is a "test" only split, and missing a "train" split, we remap it as a "train" split
        if "train" not in src_dataset.keys():
            if "test" in src_dataset.keys():
                src_dataset["train"] = src_dataset["test"]
                del src_dataset["test"]
            else:
                raise ValueError('Dataset must have a "train" split')

        # If an int value is used, it is interprated as document count
        # If a floating value (<1.0) is used, it is interprated as a percentage of the dataset
        if kargs["dataset_offset"] > 0 or kargs["dataset_length"] > 0:
            # src dataset length
            train_length = len(src_dataset["train"])

            # Compute the offset position
            offset_val = kargs["dataset_offset"]

            # If offset is a float, we will use it as a percentage
            if offset_val < 0:
                offset_val = 0
            if offset_val > 0 and offset_val < 1.0:
                offset_val = int(train_length * offset_val) # Rounded down value

            # Compute the length position
            length_val = kargs["dataset_length"]
            if length_val < 0:
                length_val = train_length - offset_val
            if length_val > 0 and length_val < 1.0:
                length_val = int(train_length * length_val)
            if length_val > (train_length - offset_val):
                length_val = (train_length - offset_val)

            # Get the subset of the dataset
            src_dataset["train"] = src_dataset["train"].select(range(offset_val, offset_val + length_val))

        # Tokenizer vars
        hf_tokenizer = None
        world_tokenizer = None

        # Load the tokenizer according to either its predefined name or its path
        # (defaults to neox)
        if kargs["tokenizer"] == "neox":
            tokenizer_file = os.path.join(SRC_DIR, "./dataflow/20B_tokenizer.json")
            hf_tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file)
        elif kargs["tokenizer"] == "world":
            # Setup the tokenizer
            world_tokenizer = True
        else:
            # AutoTokenizer
            tokenizerName = kargs["tokenizer"]

            # with custom args and props
            tokenizerKWArgs = {}
            tokenizerProps = {}
            if kargs["autoTokenizer"] is not None:
                if kargs["autoTokenizer"]["kwargs"] is not None:
                    tokenizerKWArgs = kargs["autoTokenizer"]["kwargs"]
                if kargs["autoTokenizer"]["props"] is not None:
                    tokenizerProps  = kargs["autoTokenizer"]["props"]

            # Intialize the tokenizer, with kwargs
            hf_tokenizer = AutoTokenizer.from_pretrained(tokenizerName, **tokenizerKWArgs)

            # Configure the tokenizer properties
            for k, v in tokenizerProps.items():
                setattr(hf_tokenizer, k, v)

        # Function used to tokenize the dataset as per HF tokenizer format
        # if given the textual data, it will return the tokenized data
        def encodeTokens(x):
            if world_tokenizer is True:
                # If x is an array of strings, we encode them seperately, and conslidate the result
                if isinstance(x, list):
                    id_arr = []
                    type_arr = []
                    mask_arr = []
                    for i in range(len(x)):
                        enc_str = world_tokenizer_encode(x[i], world_add_endoftext_token=world_add_endoftext_token)
                        id_arr.append(enc_str)
                        type_arr.append([0] * len(enc_str))
                        mask_arr.append([1] * len(enc_str))

                    # Consolidate the result
                    return {
                        'input_ids': id_arr,
                        'token_type_ids': type_arr,
                        'attention_mask': mask_arr
                    }

                # Else we encode the string and return it following the HF tokenizer format
                enc_str = world_tokenizer_encode(x, world_add_endoftext_token=world_add_endoftext_token)
                return {
                    'input_ids': enc_str,
                    'token_type_ids': [0] * len(enc_str),
                    'attention_mask': [1] * len(enc_str)
                }

            # We use the HF tokenizer as it is, and get the input_ids
            return hf_tokenizer(x)

        # Multi column merging default values setup
        if kargs["multi_column_keys"] is None:
            multi_column_keys = ['instruction', 'input', 'output']
            multi_column_prefix = ['Instruction:\n', 'Input:\n', 'Output:\n']
            multi_column_suffix = ['', '', '']
            multi_column_train_mask = [True, False, True]
            multi_column_separator = '\n\n'
        else:
            multi_column_keys = kargs["multi_column_keys"]
            multi_column_prefix = kargs["multi_column_prefix"]
            multi_column_suffix = kargs["multi_column_suffix"]
            multi_column_train_mask = kargs["multi_column_train_mask"]
            multi_column_separator = kargs["multi_column_separator"]

        # Tokenized encodings for multi column keys
        multi_column_enabled = len(multi_column_keys) > 0
        multi_column_prefix_encodings = []
        multi_column_suffix_encodings = []
        multi_column_separator_encodings = None

        # Process the multi column settings
        if multi_column_enabled:

            # Tokenize the multi column strings
            for i in range(len(multi_column_keys)):
                if multi_column_prefix is not None and multi_column_prefix[i] is not None:
                    multi_column_prefix_encodings.append(encodeTokens(multi_column_prefix[i]))
                if multi_column_suffix is not None and multi_column_suffix[i] is not None:
                    multi_column_suffix_encodings.append(encodeTokens(multi_column_suffix[i]))

            # Tokenize the multi column separator
            if multi_column_separator is not None and len(multi_column_separator) > 0:
                multi_column_separator_encodings = encodeTokens(multi_column_separator)

        # Maps the dataset record to the tokenized result
        # handles a wide variety of format according to the data configuration
        #
        # - custom text keys
        # - multiple key columns merged
        # - prompt/completion format
        # - text column itself
        #
        # Throws an error, if it failed to process the record
        #
        # This is called for each row record in the dataset
        def map_tokenizer(x):
            # Custom text column support
            if kargs["custom_text_key"] is not None:
                if kargs["custom_text_key"] in x:
                    return encodeTokens(x[kargs["custom_text_key"]])

            # Multi column merging support
            if multi_column_enabled:
                # Lets count the number of columns we have
                # that have data in them
                num_columns = 0
                for i in range(len(multi_column_keys)):
                    if multi_column_keys[i] in x and x[multi_column_keys[i]] is not None and len(x[multi_column_keys[i]]) > 0:
                        num_columns += 1
                # If we have more than 1 column, we will have to merge them
                if num_columns > 1:
                    # Array of output values we will return
                    input_ids = []
                    token_type_ids = []
                    attention_mask = []

                    # First item flag
                    is_first_item = True

                    # Lets loop through each column
                    for i in range(len(multi_column_keys)):
                        # And process the column if it has data
                        if multi_column_keys[i] in x and x[multi_column_keys[i]] is not None and len(x[multi_column_keys[i]]) > 0:
                            # Add the separator if this is not the first item
                            if not is_first_item and multi_column_separator_encodings is not None:
                                input_ids += multi_column_separator_encodings['input_ids']
                                token_type_ids += multi_column_separator_encodings['token_type_ids']
                                attention_mask += multi_column_separator_encodings['attention_mask']

                            # Add the prefix
                            if multi_column_prefix_encodings[i] is not None:
                                input_ids += multi_column_prefix_encodings[i]['input_ids']
                                token_type_ids += multi_column_prefix_encodings[i]['token_type_ids']
                                attention_mask += multi_column_prefix_encodings[i]['attention_mask']

                            # Tokenize the column
                            column_encodings = encodeTokens(x[multi_column_keys[i]])

                            # Add the column
                            input_ids += column_encodings['input_ids']
                            token_type_ids += column_encodings['token_type_ids']

                            # Override the training attention mask if masking is set to false
                            if len(multi_column_train_mask) < i and multi_column_train_mask[i] is False:
                                attention_mask += ([0] * len(column_encodings['input_ids']))
                            else:
                                attention_mask += ([1] * len(column_encodings['input_ids']))

                            # Add the suffix
                            if multi_column_suffix_encodings[i] is not None:
                                input_ids += multi_column_suffix_encodings[i]['input_ids']
                                token_type_ids += multi_column_suffix_encodings[i]['token_type_ids']
                                attention_mask += multi_column_suffix_encodings[i]['attention_mask']

                            # Set the first item flag to false
                            is_first_item = False

                    # Return the merged columns
                    return {
                        'input_ids': input_ids,
                        'token_type_ids': token_type_ids,
                        'attention_mask': attention_mask
                    }

            # Prompt completion support
            if 'prompt' in x and 'completion' in x:
                # Array of output values we will return
                input_ids = None
                token_type_ids = None
                attention_mask = None

                # Tokenize both prompt and completion
                # Note that the tokenizer will process and return the input_ids in batches
                prompt_encodings = encodeTokens(x['prompt'])
                completion_encodings = encodeTokens(x['completion'])

                # Join the two input_ids lists
                input_ids = prompt_encodings['input_ids'] + completion_encodings['input_ids']
                # Join the two token_type_ids lists
                token_type_ids = prompt_encodings['token_type_ids'] + completion_encodings['token_type_ids']
                # Setup the attention mask, 0 for prompt, 1 for completion, if masking is enabled
                if kargs["disable_prompt_completion_mask"]:
                    attention_mask = ([1] * len(prompt_encodings['input_ids']) + [1] * len(completion_encodings['input_ids']))
                else:
                    attention_mask = ([0] * len(prompt_encodings['input_ids']) + [1] * len(completion_encodings['input_ids']))

                # Prepare and return the output object
                return {
                    'input_ids': input_ids,
                    'token_type_ids': token_type_ids,
                    'attention_mask': attention_mask,
                }

            # Fallback to standard text tokenization
            if 'text' in x:
                return encodeTokens(x['text'])

            raise ValueError('Invalid dataset format, must contain either the configured "multi column" or prompt/completion or text')

        # Map the dataset to the tokenizer, removing the old text column
        src_dataset = src_dataset.map(map_tokenizer, batched=False, num_proc=num_cpus)

        # Remove all features, except input_ids, token_type_ids and attention_mask
        # as the metadata/etc columns may cause problems down the line (when passed to the trainer)
        dataset_features = src_dataset["train"].features
        dataset_features_to_remove = {k: v for k, v in dataset_features.items() if k not in ["input_ids", "token_type_ids", "attention_mask"]}
        src_dataset = src_dataset.remove_columns(list(dataset_features_to_remove.keys()))

        # Get the newline token
        newline_tokenSet = encodeTokens(["\n"])

        # See if rechunking is needed, this is useful mostly for "text" based datasets
        # where we would need to split them into "digestable" context length sizes
        # used for foundation training
        # ---

        # The rechunking function
        def rechunk_text(x):
            # Full Raw values that we will need to "rechunk"
            full_input_ids = []
            full_token_type_ids = []
            full_attention_mask = []

            # Loop through the x input, and build the raw values
            for i in range(len(x["input_ids"])):
                # Get the respective values and push them to the
                # raw value array, effectively merging the arrays together
                # with the newline token in between
                full_input_ids += x["input_ids"][i] + newline_tokenSet["input_ids"][0]
                full_token_type_ids += x["token_type_ids"][i] + newline_tokenSet["token_type_ids"][0]
                full_attention_mask += x["attention_mask"][i] + newline_tokenSet["attention_mask"][0]

            # Total length, and sample count
            # note that thte "remainder" will be discarded
            total_len = len(full_input_ids)
            total_samples = total_len // kargs["text_rechunk_size"]

            # The output arrays
            out_input_ids = []
            out_token_type_ids = []
            out_attention_mask = []

            # Generate the output arrays
            for i in range(total_samples):
                # Calculate the start and end of the sample
                start = i * kargs["text_rechunk_size"]
                end = start + kargs["text_rechunk_size"]

                # Push the sample to the output arrays
                out_input_ids.append(full_input_ids[start:end])
                out_token_type_ids.append(full_token_type_ids[start:end])
                out_attention_mask.append(full_attention_mask[start:end])

            # Prepare and return the output object
            ret = {
                'input_ids': out_input_ids,
                'token_type_ids': out_token_type_ids,
                'attention_mask': out_attention_mask,
            }
            return ret

        # Perform rechunking if needed for "text" based datasets
        if kargs["source"] == "text" and kargs["text_rechunk_size"] > 0:
            src_dataset = src_dataset.map(rechunk_text, batched=True,
                                        batch_size=kargs["text_rechunk_size"]*10,
                                        num_proc=num_cpus)

        # Remove empty datasets (it causes an error otherwise)
        # and perform min/max length filtering (if configured)
        def dataset_filter(x):
            row_length = len(x["input_ids"])
            if row_length <= 0:
                return False
            if kargs["min_token_size"] > 0 and row_length < kargs["min_token_size"]:
                return False
            if kargs["max_token_size"] > 0 and row_length > kargs["max_token_size"]:
                return False
            return True
        src_dataset = src_dataset.filter(dataset_filter, num_proc=num_cpus)

        # Perform a sort by length
        if kargs["sort_by_length"]:
            sort_asc = kargs["sort_asc"]

            def add_length(example):
                example["length"] = len(example['input_ids'])
                return example

            src_dataset = src_dataset.map(add_length)

            # sort by length (not sorting the columns, just the rows)
            src_dataset = src_dataset.sort("length", reverse=not sort_asc)

        # Perform rechunking after filtering, if source is not a "text" based
        # dataset and text_rechunk_force is enabled
        if kargs["source"] != "text" and kargs["text_rechunk_size"] > 0 and kargs["text_rechunk_force"]:
            src_dataset = src_dataset.map(rechunk_text, batched=True,
                                        batch_size=kargs["text_rechunk_size"]*2,
                                        num_proc=num_cpus)

        # Check if the dataset does not have a test split
        # and if so, perform the split
        if 'test' not in src_dataset.keys():
            test_split = kargs["test_split"]
            # The minimum test size is 1, if not we will get errors in the trainer?
            if test_split <= 0 or test_split <= 0.0:
                test_split = 1
            src_dataset = src_dataset['train'].train_test_split(
                test_size=test_split,shuffle=kargs["test_split_shuffle"],
                seed=42 #Fixed seed, to prevent train/test reshuffling between test runs
            )

        # Save the dataset to disk
        src_dataset.save_to_disk(kargs["data_path"])


class RWKVDataModule(LightningDataModule):
    def __init__(
        self,
        # load_from_disk(dataset_path) param
        data_path: str,
        # load_dataset(path) param
        source: str = None,
        # load_dataset(data_dir) param
        source_data_dir: str = None,
        # Additional dataset params
        source_dataset_params: dict = None,
        # Test split of source data, if it was not already done
        test_split: float = 0.01,
        test_split_shuffle: bool = False,
        # Text rechunking size
        text_rechunk_size: int = 4096,
        text_rechunk_force: bool = False,
        # ---
        # Tokenizer settings
        # ---
        tokenizer: str = "neox",
        autoTokenizer = None,

        # Add <|endoftext|> string token to the world tokenizer, at index 0
        # this was missing from the original world trie_tokenizer
        world_add_endoftext_token: bool = True,

        # ---
        # HF dataset conversion helpers
        # ---
        # Min / Max token size filtering
        min_token_size: int = -1,
        max_token_size: int = -1,

        # Sort by length
        sort_by_length: bool = False,
        sort_asc: bool = True,

        # Dataset offset and limit controls
        dataset_offset: int = -1,
        dataset_length: int = -1,

        # Custom 'text' column to support, mostly used for dataset where the
        # desired train data is in another column (eg. 'code')
        custom_text_key: str = None,
        # Multi column merging support, used for instruct/input/output datasets
        # or similar varients where the input and output are in different columns
        # and need to be merged
        multi_column_keys: list = None,
        multi_column_prefix: list = None,
        multi_column_suffix: list = None,
        multi_column_train_mask: list = None,
        multi_column_separator: str = None,
        # prompt/completion format masking support
        disable_prompt_completion_mask: bool = False,
        # Skip database setup checks if datapath exists, ignored if using preload_datapath.py
        skip_datapath_setup: bool = False,

        multitoken_width: int = 1,
    ):
        # Capture the init parameters
        self._init_locals = locals()
        del self._init_locals["self"]
        del self._init_locals["__class__"]

        super().__init__()
        self.data_path = data_path
        self._loaded_dataset = None

        # Log to wandb
        if wandb.run is not None:
            wandb.config.update({ "data":dict(self._init_locals) })

    # Called once for initial setup
    def prepare_data(self):
        prepare_data_static(**self._init_locals)

    # Setup process that is universal
    def _internal_setup(self):
        if self._loaded_dataset is None:
            self._loaded_dataset = load_from_disk(self.data_path).with_format('torch')

    # Called once for every process in DDP
    def setup(self, stage):
        self._internal_setup()

    # Return the train dataloader
    def train_dataloader(self):
        self._internal_setup()
        return DataLoader(self._loaded_dataset['train'], num_workers=num_workers)

    # Return the validation dataloader
    def val_dataloader(self):
        self._internal_setup()
        return DataLoader(self._loaded_dataset['test'], num_workers=num_workers)


Overwriting /content/RWKV-infctx-trainer/RWKV-v5/src/data.py


In [ ]:
#@title patch model.py

%%writefile /content/RWKV-infctx-trainer/RWKV-v5/src/model.py

########################################################################################################
# The RWKV Language Model - https://github.com/BlinkDL/RWKV-LM
########################################################################################################

import gc, math, os
from random import randint
from typing import List, Optional

import numpy as np
import torch
# torch._C._jit_set_profiling_executor(True)
# torch._C._jit_set_profiling_mode(True)
import torch.nn as nn
from torch.nn import functional as F

import lightning as L
from lightning.pytorch.utilities import rank_zero_info, rank_zero_only
from lightning.pytorch.strategies import DeepSpeedStrategy

import deepspeed
from deepspeed.ops.adam import DeepSpeedCPUAdam, FusedAdam
import deepspeed.runtime.lr_schedules
import wandb

from torch.utils.cpp_extension import load

# Script dir for various files
SCRIPT_DIR = os.path.dirname(os.path.realpath(__file__))
CUDA_DIR = os.path.abspath(os.path.join(SCRIPT_DIR, "../cuda"))

# janky hack mate
MULTITOKEN_RANGES = tuple(
    (i*1024, (i+1)*1024) for i in range(37)
)
PADDING_IDX = 2**16-1

########################################################################################################
# JIT / torch compile special handling
########################################################################################################

# Currently the features we need for torch compile, is avaliable only in
# 2.1 nightly build (and is expected to be in 2.1 official release)
#
# However because the nightly build torch.compile behaviour has been unstable
# the versioning code to check for enabling toch compile will not be used,
# until we confirm a stable version of torch.compile
from packaging import version
def is_torch_version_above(required_version):
    torch_version = version.parse(torch.__version__.split('+')[0])
    return torch_version >= version.parse(required_version)
IS_TORCH_2_1 = is_torch_version_above("2.0.9999")

# Get the JIT / torch compile option flags from the environment
RWKV_JIT_ON        = os.getenv("RWKV_JIT_ON", "1").lower() in ("1", "true", "yes")
RWKV_TORCH_COMPILE = os.getenv("RWKV_TORCH_COMPILE", f"0").lower() in ("1", "true", "yes")
RWKV_TORCH_RUN_MODE = None

# We enable JITMod*/Function when supporting torch.jit
# We use TorchCompile* when supporting torch compile
# based on the current runtime settings
if RWKV_TORCH_COMPILE:
    RWKV_TORCH_RUN_MODE = "torch-compile"

    JITModClass  = nn.Module
    JITModMethod = lambda x: x
    JITFunction  = lambda x: x

    # PS: i have tried mode="max-autotune", and mode="reduce-overhead", however they crash
    #     for now (8th July 2023). I may introduce them in the future once they are stable
    #
    #     Additionally, torch.compile has issues with the pytorch.lightning module directly
    # ---

    # We generally have 2 major options, either we use torch.compile
    # onto the key top level functions (train, val, test, predict, etc)
    # and let the compiler handle all the decision making on how to optimize
    #
    # However this was found to basically just match JIT level of performance exactly
    # ---
    # TCompileMax          = lambda x: x
    # TCompileBaseline     = lambda x: torch.compile(x, fullgraph=False)

    # Alternatively, we can perform a much more aggressive optimization on critical functions
    # that we know are compatible with torch.compile(fullgraph=True) - which provides the highest
    # level of optimization possible with torch.compile
    # ---
    TCompileMax        = lambda x: torch.compile(x, fullgraph=True)
    TCompileBaseline   = lambda x: x

    # ---
    # Because torch.compile is expected to change overtime, the two options should
    # be tested every now and then, for any performance changes
    #
    # and we should switch over to the broaded automated approach if its "faster"
    # ---

    # Used to wrap functions which are **not** torch.compile compatible
    TCompileDisable    = torch._dynamo.disable

    # The following are known warnings in the nightly build, that can be safely ignored for stable release
    #
    # `torch._inductor.utils: [WARNING] DeviceCopy in input program`
    # https://discuss.pytorch.org/t/what-can-cause-warning-devicecopy-in-input-program/175566

elif RWKV_JIT_ON:
    RWKV_TORCH_RUN_MODE = "torch-jit"
    JITModClass  = torch.jit.ScriptModule
    JITModMethod = torch.jit.script_method
    JITFunction  = torch.jit.script

    # JITModClass  = nn.Module
    # JITModMethod = lambda x: x
    # JITFunction  = lambda x: x

    TCompileMax        = lambda x: x
    TCompileBaseline   = lambda x: x
    TCompileDisable    = lambda x: x
else:
    RWKV_TORCH_RUN_MODE = "torch-native"
    JITModClass  = nn.Module
    JITModMethod = lambda x: x
    JITFunction  = lambda x: x

    TCompileMax        = lambda x: x
    TCompileBaseline   = lambda x: x
    TCompileDisable    = lambda x: x

print(f"[RWKV.model] Running RWKV model using '{RWKV_TORCH_RUN_MODE}' with torch '{torch.__version__}'")

# ---
# Isolating out known operations that **does not work** with torch.compile
# and wrapping them within a torch._dynamo.disable, this is required to get
# the baseline torc.compile to work
# ---

@TCompileDisable
def deepspeed_checkpoint(*args, **kwargs):
    return deepspeed.checkpointing.checkpoint(*args, **kwargs)

########################################################################################################
# RWKV: State Blocks
########################################################################################################

class TimeMixState:

    def __init__(self, shift_state: torch.Tensor, wkv_state: torch.Tensor):
        self.shift_state = shift_state
        self.wkv_state = wkv_state


class ChannelMixState:

    def __init__(self, shift_state: torch.Tensor):
        self.shift_state = shift_state


class BlockState:

    def __init__(self, time_mix_state: TimeMixState,
                 channel_mix_state: ChannelMixState):
        self.time_mix_state = time_mix_state
        self.channel_mix_state = channel_mix_state


class BlockStateList:

    def __init__(self, shift_states, wkv_states):
        self.wkv_states = wkv_states
        self.shift_states = shift_states

    # @ TCompileMax (no difference)
    @staticmethod
    def create(N, B, C, n_head, head_size, device, dtype):
        result = BlockStateList.empty(N, B, C, n_head, head_size, device, dtype)
        result.wkv_states[:] = 0
        # result.wkv_states[:, :, :, -1] = -1e38
        result.shift_states[:] = 0
        return result

    # @ TCompileMax (no difference)
    @staticmethod
    def empty(N, B, C, n_head, head_size, device, dtype):
        # @TODO: confirm if dtype can be changed from .flaot to dtype=dtype (when bf16)
        wkv_states = torch.empty((N, B, n_head, head_size, head_size),
                                 device=device,
                                #  dtype=dtype)
                                 dtype=torch.float)
        shift_states = torch.empty((N, 2, B, C), device=device, dtype=dtype)
        return BlockStateList(shift_states, wkv_states)

    def __getitem__(self, layer: int):
        return BlockState(
            TimeMixState(self.shift_states[layer, 0], self.wkv_states[layer]),
            ChannelMixState(self.shift_states[layer, 1]))

    def __setitem__(self, layer: int, state: BlockState):
        self.shift_states[layer, 0] = state.time_mix_state.shift_state
        self.wkv_states[layer] = state.time_mix_state.wkv_state
        self.shift_states[layer, 1] = state.channel_mix_state.shift_state

########################################################################################################
# RWKV: RWKV Time-mix + RWKV Channel-mix
########################################################################################################

class RWKV_TimeMix(JITModClass):

    def __init__(self, layer_id, n_layer, n_embd, n_head, head_size, dim_att):
        super().__init__()

        self.dim_att = dim_att
        self.n_layer = n_layer
        self.n_embd = n_embd
        self.layer_id = layer_id

        self.n_head = n_head
        self.head_size = head_size

        # Optimized chunk length is fixed for now
        self.chunk_len = 512
        # assert ctx_len % self.chunk_len == 0

        with torch.no_grad():  # fancy init
            ratio_0_to_1 = layer_id / (n_layer - 1)  # 0 to 1
            ratio_1_to_almost0 = 1.0 - (layer_id / n_layer)  # 1 to ~0
            ddd = torch.ones(1, 1, n_embd)
            for i in range(n_embd):
                ddd[0, 0, i] = i / n_embd

            # fancy time_mix
            self.time_mix_k = nn.Parameter(torch.pow(ddd, ratio_1_to_almost0))
            self.time_mix_v = nn.Parameter(torch.pow(ddd, ratio_1_to_almost0) + 0.3 * ratio_0_to_1)
            self.time_mix_r = nn.Parameter(torch.pow(ddd, 0.5 * ratio_1_to_almost0))

            # fancy time_decay
            decay_speed = torch.ones(n_head)
            for h in range(n_head):
                decay_speed[h] = -8 + 7 * (h / (n_head - 1)) ** (0.7 + 1.3 * ratio_0_to_1)
            self.time_decay = nn.Parameter(decay_speed)
            # print(layer_id, self.time_decay.flatten()[:3].cpu().numpy(), '...', self.time_decay.flatten()[-3:].cpu().numpy())

            # V5-R2 changes
            self.time_faaaa = nn.Parameter(torch.ones(n_head) * 0.05)
            # self.time_first = nn.Parameter(torch.ones(n_head) * (-3.0))

        # self.time_shift = nn.ZeroPad2d((0, 0, 1, -1))
        self.receptance = nn.Linear(n_embd, dim_att, bias=False)
        self.key = nn.Linear(n_embd, dim_att, bias=False)
        self.value = nn.Linear(n_embd, dim_att, bias=False)
        self.output = nn.Linear(dim_att, n_embd, bias=False)

        self.ln_x = nn.GroupNorm(n_head, n_embd)

    # this is based on jit_func(self,x)
    @JITModMethod
    def _forward_rkv_chunk(self, x, B, TT, last_state: TimeMixState):
        # Mix x with the previous timestep to produce xk, xv, xr
        xx = torch.concat((last_state.shift_state.unsqueeze(1), x[:, :-1]), dim=1)
        # xx = self.time_shift(x)

        xk = x * self.time_mix_k + xx * (1 - self.time_mix_k)
        xv = x * self.time_mix_v + xx * (1 - self.time_mix_v)
        xr = x * self.time_mix_r + xx * (1 - self.time_mix_r)

        r = self.receptance(xr).view(B, TT, self.n_head, self.head_size).transpose(1, 2)            # BTC -> BHTS
        k = self.key(xk).view(B, TT, self.n_head, self.head_size).transpose(1, 2).transpose(-2, -1) # BTC -> BHTS -> BHST
        v = self.value(xv).view(B, TT, self.n_head, self.head_size).transpose(1, 2)                 # BTC -> BHTS

        return r, k, v

    def _forward_wkbs_chunk(self, T, r, k, v):
        H = self.n_head

        w = torch.exp(-torch.exp(self.time_decay.float())).unsqueeze(-1)

        # V5-R2 changes
        u = self.time_faaaa.float().unsqueeze(-1)
        # u = torch.exp(self.time_first.float()).unsqueeze(-1)

        ws = w.pow(T).reshape(1, H, 1, 1)
        ind = torch.arange(T-1, -1, -1, device=r.device).unsqueeze(0).repeat(H, 1)
        w = w.repeat(1, T).pow(ind)

        wk = w.reshape(1, H, 1, T)
        wb = wk.transpose(-2, -1).flip(2)

        w = torch.cat([w[:, 1:], u], dim=1)
        w = F.pad(w, (0, T))
        w = torch.tile(w, [T])
        w = w[:, :-T].reshape(-1, T, 2 * T - 1)
        w = w[:, :, T-1:].reshape(1, H, T, T)

        w = w.to(dtype=r.dtype)
        wk = wk.to(dtype=r.dtype)
        wb = wb.to(dtype=r.dtype)
        ws = ws.to(dtype=r.dtype)

        return w, wk, wb, ws

    @JITModMethod
    def _forward_state_chunk(self, r, k, v, w, wk, wb, ws, x_l, last_state: TimeMixState):
        B, H, TT, S = r.size()
        T = TT

        # s = torch.zeros(B, H, S, S, device=r.device, dtype=r.dtype)  # state
        s = last_state.wkv_state

        if r.dtype == torch.bfloat16 and s.dtype != torch.bfloat16:
            s = s.contiguous().to(torch.bfloat16)

        x = torch.zeros(B, H, TT, S, device=r.device, dtype=r.dtype) # output

        ########################################################################
        for i in range(TT // T):

            rr = r[:, :, i*T:i*T+T, :]
            kk = k[:, :, :, i*T:i*T+T]
            vv = v[:, :, i*T:i*T+T, :]

            x[:, :, i*T:i*T+T, :] = ((rr @ kk) * w) @ vv  +  (rr @ s) * wb

            s = ws * s + (kk * wk) @ vv
        ########################################################################

        x = x.transpose(1, 2).contiguous().view(B * TT, H*S) # BHTS -> BTHS -> BTC
        x = self.ln_x(x/8).view(B, TT, H*S)

        return self.output(x), TimeMixState(x_l, s)

    def _forward_chunk(self, x, last_state: TimeMixState):
        # Forward sizings (Batch, Time/ContextLength, Tokens)
        B, TT, C = x.size()
        B = torch.tensor(B, device=x.device, dtype=torch.int32)
        TT = torch.tensor(TT, device=x.device, dtype=torch.int32)

        # Get r, k, v (self.jit_func(x))
        r, k, v = self._forward_rkv_chunk(x, B, TT, last_state)

        # Get w, wk, wb, ws (self.jit_func_2)
        w, wk, wb, ws = self._forward_wkbs_chunk(TT, r, k, v)

        # Does the state forwarding
        return self._forward_state_chunk(r, k, v, w, wk, wb, ws, x[:, -1], last_state)

    @TCompileMax
    def forward(self, x, last_state: TimeMixState):
        # Get the x sizing
        B, TT, C = x.size()

        # Chunk length to split by,
        # we probably can reoptimize this at some point
        chunk_len = self.chunk_len

        # Logits to return
        x_logits = torch.zeros(B, TT, C, device=x.device, dtype=x.dtype)

        # Split the input by TT chunks
        for i in range(0, TT, chunk_len):
            x_chunk = x[:, i:i+chunk_len, :]
            chunk_logits, last_state = self._forward_chunk(x_chunk, last_state)
            x_logits[:, i:i+chunk_len, :] = chunk_logits

        # Return the logits and the state
        return x_logits, last_state


########################################################################################################


class RWKV_ChannelMix(JITModClass):

    def __init__(self, layer_id, n_layer, n_embd, dim_ffn):
        super().__init__()

        with torch.no_grad():  # fancy init of time_mix
            ratio_1_to_almost0 = 1.0 - (layer_id / n_layer)  # 1 to ~0
            ddd = torch.ones(1, 1, n_embd)
            for i in range(n_embd):
                ddd[0, 0, i] = i / n_embd
            self.time_mix_k = nn.Parameter(torch.pow(ddd, ratio_1_to_almost0))
            self.time_mix_r = nn.Parameter(torch.pow(ddd, ratio_1_to_almost0))

        self.key = nn.Linear(n_embd, dim_ffn, bias=False)
        self.receptance = nn.Linear(n_embd, n_embd, bias=False)
        self.value = nn.Linear(dim_ffn, n_embd, bias=False)

    @JITModMethod
    @TCompileMax
    def forward(self, x, last_state: ChannelMixState):
        xx = torch.concat((last_state.shift_state.unsqueeze(1), x[:, :-1]),
                          dim=1)
        xk = x * self.time_mix_k + xx * (1 - self.time_mix_k)
        xr = x * self.time_mix_r + xx * (1 - self.time_mix_r)
        k = self.key(xk)
        k = torch.square(torch.relu(k))
        kv = self.value(k)
        return (torch.sigmoid(self.receptance(xr)) * kv,
                ChannelMixState(x[:, -1]))


########################################################################################################
# The RWKV Model blocks
########################################################################################################

class Block(nn.Module):

    def __init__(self, layer_id, n_layer, n_embd, n_head, head_size, dropout, dim_att, dim_ffn):
        super().__init__()
        self.layer_id = layer_id

        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

        if self.layer_id == 0:
            self.ln0 = nn.LayerNorm(n_embd)

        self.att = RWKV_TimeMix(layer_id, n_layer, n_embd, n_head, head_size, dim_att)
        self.ffn = RWKV_ChannelMix(layer_id, n_layer, n_embd, dim_ffn)

        # Setup droupout at block level
        self.dropout = dropout
        if dropout > 0:
            self.drop0 = nn.Dropout(p = dropout)
            self.drop1 = nn.Dropout(p = dropout)

    def forward(self, x, last_state: BlockState):
        if self.layer_id == 0:
            x = self.ln0(x)

        att_out, att_state = self.att(
            self.ln1(x),
            last_state.time_mix_state,
        )

        if self.dropout > 0.0:
            # Handle with dropout
            x = self.drop0(x + att_out)
            ffn_out, ffn_state = self.ffn(
                self.ln2(x),
                last_state.channel_mix_state,
            )
            x = self.drop1(x + ffn_out)
        else:
            # Handle without dropout
            x = x + att_out
            ffn_out, ffn_state = self.ffn(
                self.ln2(x),
                last_state.channel_mix_state,
            )
            x = x + ffn_out

        return x, BlockState(att_state, ffn_state)


class L2Wrap(torch.autograd.Function):

    @staticmethod
    def forward(ctx, loss, y, token_amount, currentMask):
        # Currently (8th July 2023), save_for_backward, causes an issue with
        # pytorch.compile (see: https://github.com/pytorch/pytorch/blob/e600505e3209eaf539e8bc99870ea55236cefbf5/torch/_dynamo/variables/higher_order_ops.py#L735)
        #
        # Due to L2Wrap being a major hotspot, we should monitor this for future support.
        # so that once its resolved, we can include the L2Wrap step in the torch.compile path
        #
        # See also:
        # - checkpointed_step
        ctx.save_for_backward(y)
        ctx.token_amount = token_amount
        ctx.currentMask = currentMask
        return loss

    @staticmethod
    def backward(ctx, grad_output):
        y, = ctx.saved_tensors
        token_amount = ctx.token_amount
        # to encourage the logits to be close to 0
        factor = 1e-4 / token_amount
        maxx, ids = torch.max(y, -1, keepdim=True)
        gy = torch.zeros_like(y)
        gy.scatter_(-1, ids, maxx * factor)
        gy = gy * ctx.currentMask[:, None][None, :]
        return (grad_output, gy, None, None)

########################################################################################################
# Static optimized functions
########################################################################################################

# @ TCompileMax (no speed improvement)
# def F_cross_entropy_reduction_none_optimized(logits, targets):
#     return F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), reduction="none")

########################################################################################################
# Core RWKV module
########################################################################################################
class RWKV(L.LightningModule):

    def __init__(self,
                 # Model file path to load from
                 load_model: str,
                 # Model size settings, which we either
                 # "auto detect", or use the user specified settings
                 n_embd: int = -1,
                 n_layer: int = -1,
                 vocab_size: int = -1,
                 # Context length size for the model
                 ctx_len: int = 2048,
                 # Context length schedule
                 ctx_len_cutoffs: List[int] = [],
                 ctx_len_warmup_steps: List[int] = [],
                 # Learning rate schedule
                 # use only target_lr_init / lr_init
                 # to configure a constant learning rate
                 lr_init: float = -1.0,
                 lr_final: float = -1.0,
                 lr_period: int = -1,
                 lr_period_type: str = 'epoch',
                 # Dropout rate
                 dropout: float = 0.0,
                 # Adam optimizer settings
                 beta1: float = 0.9,
                 beta2: float = 0.99,
                 adam_eps: float = 1.0e-08,
                 weight_decay: float = 0.01,
                 warmup_steps: int = -1,
                 # loss bias start
                 position_loss_bias: float = 1.0,
                 position_loss_bias_in_validation: bool = False,
                 # Backprop settings
                 grad_cp: bool = True,
                 bptt_learning: bool = True,
                 bptt_learning_range: int = -1,
                 bptt_truncated_learning: bool = False,
                 layerwise_lr: bool = True,
                 dim_att: Optional[int] = None,
                 dim_ffn: Optional[int] = None,
                 substep_cuda_cache_clear: bool = False,
                 substep_logging: bool = False,
                 torch_set_float32_matmul_precision:str = 'high'
                 ):

        # Lets save everything in one shot
        # (this is used for wandb logging)
        self.setup_args = locals()
        del self.setup_args["self"]
        del self.setup_args["__class__"]

        # Setup the parent class
        super().__init__()

        # Load the model, unless its the special ".//<#|=@%!$init_model$!%@=|#>//." path
        # which is reserved to be used with the `init_model.py`
        #
        # We intentionally used several filesystem illegal characters, to ensure it
        # is not accidentally used by the user for a real file
        model_weights = None
        model_keys = None
        if load_model != ".//<#|=@%!$init_model$!%@=|#>//.":
            # Check if the load_model path exists, and is a file
            if not os.path.isfile(load_model):
                raise ValueError(f"load_model file '{load_model}' does not exist")

            # Load the model weights
            model_weights = torch.load(load_model, map_location='cpu')

            # Get the model keys
            model_keys = list(model_weights.keys())

        # Lets compute the model various sizes, if they are not provided
        if n_layer < 0:
            max_block_id = 0
            for x in model_keys:
                if 'blocks.' in x:
                    block_id = int(x.split('.')[1])
                    max_block_id = max(max_block_id, block_id)
            n_layer = max_block_id + 1

        if n_embd < 0:
            n_embd = model_weights['head.weight'].shape[1]

        if vocab_size < 0:
            vocab_size = model_weights['head.weight'].shape[0]

        # Save the various other params for later
        self.ctx_len = ctx_len
        self.ctx_len_cutoffs = ctx_len_cutoffs
        self.ctx_len_warmup_steps = ctx_len_warmup_steps
        self.n_embd = n_embd
        self.n_layer = n_layer
        self.vocab_size = vocab_size
        self.layerwise_lr = layerwise_lr
        self.grad_cp = grad_cp
        self.lr_init = lr_init
        self.lr_final = lr_final
        self.lr_period = lr_period
        self.lr_period_type = lr_period_type
        self.dropout = dropout
        self.warmup_steps = warmup_steps
        self.beta1 = beta1
        self.beta2 = beta2
        self.weight_decay = weight_decay
        self.adam_eps = adam_eps
        self.bptt_learning = bptt_learning
        self.bptt_learning_range = bptt_learning_range
        self.bptt_truncated_learning = bptt_truncated_learning
        self.substep_cuda_cache_clear = substep_cuda_cache_clear
        self.substep_logging = substep_logging

        # Save the position loss params
        self.position_loss_bias = position_loss_bias
        self.position_loss_bias_in_validation = position_loss_bias_in_validation

        dim_att = dim_att or n_embd
        dim_ffn = dim_ffn or n_embd * 4
        self.dim_att = dim_att
        self.dim_ffn = dim_ffn

        # Compute the RWKV-v5 n_head / headsize
        head_size = 64
        n_head = n_embd // head_size
        assert n_embd % n_head == 0 ,  f"n_embd must be divisible by head_size ({self.head_size})"
        self.n_head = n_head
        self.head_size = head_size

        # Matmu precision check
        if torch_set_float32_matmul_precision is not None:
            torch.set_float32_matmul_precision(torch_set_float32_matmul_precision)

        self.emb = nn.EmbeddingBag(vocab_size, n_embd, padding_idx=PADDING_IDX)

        # load(name=f"wkv_{self.ctx_len}_bf16",
        #      sources=[
        #         os.path.join(CUDA_DIR, "wkv_op_bf16.cpp"),
        #         os.path.join(CUDA_DIR, "wkv_cuda_bf16.cu")
        #     ],
        #      verbose=True,
        #      extra_cflags=["-std=c++17", "-O3", f"-DTmax={self.ctx_len}"],
        #      extra_cuda_cflags=[
        #          "-t 4", "-std=c++17", "-res-usage", "--maxrregcount 60",
        #          "--use_fast_math", "-O3", "-Xptxas -O3",
        #          "--extra-device-vectorization", f"-DTmax={self.ctx_len}"
        #      ],
        #      is_python_module=False)

        self.blocks = nn.ModuleList([
            Block(i, n_layer, n_embd, n_head, head_size, dropout, dim_att, dim_ffn) for i in range(n_layer)
        ])

        self.ln_out = nn.LayerNorm(n_embd)
        self.head = nn.Linear(n_embd, vocab_size, bias=False)

        # Dropout handling
        if dropout > 0:
            self.drop0 = nn.Dropout(p = dropout)

        # load the state, and GC the original cpu copy
        if model_weights != None:
            self.load_state_dict(model_weights)
            del model_weights
            gc.collect()

    def configure_optimizers(self):
        if self.bptt_learning == False:
            if self.deepspeed_stage >= 2 or self.deepspeed_offload:
                print(f"[WARNING]: it is highly recommended to enable bptt_learning when used to deepspeed 2/3/offloading, otherwise an exception will occur when training with dataset records, larger then the configured context length ({self.ctx_len})")
        else:
            if self.trainer.num_devices > 1:
                if self.bptt_learning_range <= 0:
                    print("[WARNING]: unlimited bptt_learning_range across multiple GPU's has a performance penalty with datasets of mixed sizes due to its constant need to keep all GPU's in sync (consider using bptt_learning_range=1 instead)")

        # Get the learning rate used for the optimizer
        lr_init = self.lr_init
        lr_final = self.lr_final
        # If the final learning rate is not specified, use the initial learning rate
        if lr_final < 0:
            lr_final = self.lr_init

        # Log the learning rate, and various other parameters
        if self.trainer.local_rank == 0:

            # Add the important notes, for informing users of common gotchas
            print((
                "#\n"
                "# RWKV lighting_trainer.py important notes \n"
                "# https://github.com/RWKV/RWKV-infctx-trainer \n"
                "#\n"
                "# - Ensure your host is not running cuda 12.0 (use either 11.8, or >=12.1), as this is known to have freeze issues\n"
                "# - The terms used in wandb / the progress bar can be confusing, see the github README.md for beter clarifications\n"
                "# - When resuming from checkpoint, the estimated time is inaccurate\n"
                "#"
            ))

            lr_init_e = "{:.3e}".format(lr_init)
            lr_final_e = "{:.3e}".format(lr_final)
            print(f"\n[RWKV.model] Configuring optimizer with\n"+
                  f"    - lr_init:  {lr_init_e} ({lr_init})\n"+
                  f"    - lr_final: {lr_final_e} ({lr_final})\n")

            # Get the setup args
            model_args = dict(self.setup_args)
            model_args["__lr_init"] = lr_init
            model_args["__lr_final"] = lr_final

            # Update WANDB
            if wandb.run is not None:
                wandb.config.update({ "model": model_args })

        # Setup layerwise learning rate
        if self.layerwise_lr:
            lr_1x = set()
            lr_2x = set()
            lr_3x = set()
            for n, p in self.named_parameters():
                if "time_mix" in n:
                    lr_1x.add(n)
                elif "time_decay" in n:
                    lr_2x.add(n)
                # V5-R2 changes
                elif "time_faaaa" in n:
                    lr_2x.add(n)
                # elif "time_first" in n:
                #     lr_3x.add(n)
                else:
                    lr_1x.add(n)
            lr_1x = sorted(list(lr_1x))
            lr_2x = sorted(list(lr_2x))
            lr_3x = sorted(list(lr_3x))
            # print('1x', lr_1x)
            # print('2x', lr_2x)
            # print('3x', lr_3x)
            param_dict = {n: p for n, p in self.named_parameters()}
            optim_groups = [
                {
                    "params": [param_dict[n] for n in lr_1x],
                    "weight_decay": 0.0,
                    "lr": 1.0 * lr_init
                },
                {
                    "params": [param_dict[n] for n in lr_2x],
                    "weight_decay": 0.0,
                    "lr": 2.0 * lr_init
                },
                {
                    "params": [param_dict[n] for n in lr_3x],
                    "weight_decay": 0.0,
                    "lr": 3.0 * lr_init
                },
            ]
        else:
            optim_groups = [
                {
                    "params": [p for n, p in self.named_parameters()],
                    "weight_decay": 0.0
                },
            ]

        # Setup the adam optimizers
        if self.deepspeed_offload:
            optimizer = DeepSpeedCPUAdam(optim_groups,
                                         lr=lr_init,
                                         betas=(self.beta1, self.beta2),
                                         eps=self.adam_eps,
                                         bias_correction=True,
                                         adamw_mode=False,
                                         weight_decay=self.weight_decay,
                                         amsgrad=False)
        else:
            optimizer = FusedAdam(optim_groups,
                                  lr=lr_init,
                                  betas=(self.beta1, self.beta2),
                                  eps=self.adam_eps,
                                  bias_correction=True,
                                  adam_w_mode=False,
                                  weight_decay=self.weight_decay,
                                  amsgrad=False)

        # Throw if wramup_steps and lr_period are both set (not supported)
        if self.warmup_steps > 0 and self.lr_period > 0:
            raise ValueError(
                "Use either warmup_steps or lr_period, not both.")

        if self.warmup_steps > 0:
            lr_scheduler = deepspeed.runtime.lr_schedules.WarmupLR(
                optimizer,
                warmup_min_lr=0.2 * self.lr_init,
                warmup_max_lr=self.lr_init,
                warmup_num_steps=self.warmup_steps,
                warmup_type='linear')

            return {
                'optimizer': optimizer,
                'lr_scheduler': lr_scheduler,
            }

        else:
            # Skip the lr_scheduler process if lr_init and lr_final are the same
            if lr_init == lr_final:
                return optimizer

            # The total number of steps to perform training rate decay with
            lr_total_step = 0

            # Handle lr_period -1 default behaviour of using the max_step / max_epoch
            if self.lr_period == -1:
                # Get trainer max_step / max_epoch
                trainer_max_step = self.trainer.max_steps
                trainer_max_epoch = self.trainer.max_epochs
                if trainer_max_step > 0:
                    lr_total_step = trainer_max_step
                elif trainer_max_epoch > 0:
                    lr_total_step = trainer_max_epoch * self.num_step_per_epoch()
                else :
                    print("Warning: max_step/max_epoch not set, we would be performing lr_init to lr_final shift assuming 10 epoch")
                    lr_total_step = 10 * self.num_step_per_epoch()
            else:
                # Calculate lr_total_step based on lr_period
                if self.lr_period_type == "step":
                    lr_total_step = self.lr_period
                elif self.lr_period_type == "epoch":
                    lr_total_step = self.lr_period * self.num_step_per_epoch()
                else:
                    raise ValueError(f"lr_period_type {self.lr_period_type} not supported.")

            # Lets initialize the lr_scheduler
            lr_scheduler = torch.optim.lr_scheduler.LinearLR(
                optimizer,
                start_factor=1.0,
                end_factor= lr_final / lr_init,
                total_iters=lr_total_step
            )

            return {
                'optimizer': optimizer,
                'lr_scheduler': {
                    "scheduler": lr_scheduler,
                    "interval": "step",
                    "frequency": 1,
                },
            }


    # We have to compute the number of steps per epoch ourselves
    # as this value is not provided directly by pytorch lightning
    # https://github.com/Lightning-AI/lightning/issues/5449#issuecomment-1501597319
    def num_step_per_epoch(self) -> int:
        # Estimated number of steps in total, added as the following
        # https://github.com/Lightning-AI/lightning/pull/11599
        #
        # This MUST be called before len(self.trainer.train_loader)
        # otherwise there is a bug in which the train_dataloader is not
        # fully initialized, which seems to be resolved by computing the
        # self.trainer.estimated_stepping_batches
        estimated_stepping_batches = self.trainer.estimated_stepping_batches

        # Get the number of epochs,
        # use estimated_stepping_batches if max_epochs is set
        max_epochs = self.trainer.max_epochs
        if max_epochs > 0:
            return estimated_stepping_batches // max_epochs

        # Get the train_dataloader
        train_dataloader = self.trainer.train_dataloader
        if train_dataloader is None:
            train_dataloader = self.trainer.fit_loop._data_source.dataloader()

        # Max epoch is not set, use the train_dataloader
        dataset_size = len(train_dataloader)

        num_devices = max(1, self.trainer.num_devices)
        num_steps = dataset_size // (self.trainer.accumulate_grad_batches * num_devices)
        return num_steps

    @property
    def deepspeed_offload(self) -> bool:
        strategy = self.trainer.strategy
        if isinstance(strategy, DeepSpeedStrategy):
            cfg = strategy.config["zero_optimization"]
            return "offload_optimizer" in cfg or "offload_parameters" in cfg
        return False

    @property
    def deepspeed_stage(self) -> int:
        strategy = self.trainer.strategy
        if isinstance(strategy, DeepSpeedStrategy):
            cfg = strategy.config["zero_optimization"]
            return "stage" in cfg
        return -1

    @TCompileBaseline
    def forward(self, idx: torch.Tensor, last_shift_states: torch.Tensor = None,
                last_wkv_states: torch.Tensor = None):
        B, T, _ = idx.size()
        assert T <= self.ctx_len, "Cannot forward, model ctx_len is exhausted."

        x = torch.stack([self.emb(batch_) for batch_ in idx])

        # Handle dropout (input)
        if self.dropout > 0.0:
            x = self.drop0(x)

        new_states = BlockStateList.empty(self.n_layer, B, self.n_embd,
                                          self.n_head, self.head_size,
                                          x.device, x.dtype)

        # last_shift_states can be None, when we are performing direct inference
        if last_shift_states is None:
            cur_bs_list = BlockStateList.create(
                self.n_layer, B, self.n_embd,
                self.n_head, self.head_size,
                x.device, x.dtype
            )
        else:
            cur_bs_list = BlockStateList(last_shift_states, last_wkv_states)

        # Avoid using the zip operation, as torch.compile throws an exception on it
        # with `zip not reconized as a valid function`
        # ---
        # for i, (block, last_state) in enumerate(
        #         zip(self.blocks,
        #             BlockStateList(last_shift_states, last_wkv_states))):
        # ---
        for i in range(len(self.blocks)):
            block = self.blocks[i]
            last_state = cur_bs_list[i]
            if self.grad_cp:
                x, new_state = deepspeed_checkpoint(
                    block, x, last_state)
            else:
                x, new_state = block(x, last_state)
            new_states[i] = new_state

        x = self.ln_out(x)

        x = self.head(x)

        return x, new_states.shift_states, new_states.wkv_states

    #
    # Custom overwrite of manual_backwards operation, to skip the "manual_backwards"
    # safety check, so we can perform manual backward operation step, while using
    # the default trainer loop. This is modified from the original code found here:
    # https://github.com/Lightning-AI/lightning/blob/37c244f94be365496def82870b22c2faf0ab889e/src/lightning/pytorch/core/module.py#L999
    #
    # ---
    #
    # This allow us to avoid disabling the "automatic_optimization" flag
    #
    # Which would have been required to do "segmented learning", or "Backpropagation Through Time"
    # where we would need to implement manual optimization as per
    # https://lightning.ai/docs/pytorch/stable/model/manual_optimization.html
    #
    # Otherwise an error will be thrown if we call `self.manual_backward`
    #
    # However this would mean that we would need to do a full reimplementation
    # of several features that were handled by the automatic optimization.
    # - accumulate_grad_batches
    # - gradient_clip_val
    # - logging behaviour
    # - distributed training co-ordination
    # - (And probably other features that I am not aware of)
    #
    # So this is a hacky work around, to avoid reimplementing all of the above.
    #
    # From the current code implementatiion, it seem like this is blocked only by
    # automatic_optimization flag - and has no adverse side effect otherwise
    # https://lightning.ai/docs/pytorch/stable/_modules/lightning/pytorch/core/module.html#LightningModule.manual_backward
    #
    # If anyone have a better idea, let me know
    # (have experimented with, reimplementing the above, but it is not trivial, unfortunately)
    #
    def manual_backward(self, loss: torch.Tensor, *args, **kwargs):
        if self._fabric:
            self._fabric.backward(loss, *args, **kwargs)
        else:
            # self._verify_is_manual_optimization("manual_backward")
            self.trainer.strategy.backward(loss, None, *args, **kwargs)

    #
    # Main compute_loss function, this is called by the trainer loop
    #
    def compute_loss(self, batch, batch_idx, is_training_run: bool):
        seq = batch['input_ids']
        assert isinstance(seq, torch.Tensor) and seq.ndim == 3
        ori_seq_mask = batch['attention_mask']

        # Check if attent mask is set, if not initialize it
        if ori_seq_mask is None or ori_seq_mask.ndim != 2:
            ori_seq_mask = torch.ones_like(seq[:, 1:])

        # Get the starting and ending loss bias
        loss_bias_start = self.position_loss_bias
        loss_bias_end   = 2.0 - loss_bias_start

        # total_mask_sum
        total_mask_sum = torch.sum(ori_seq_mask)

        # Skip loss bias calculation, if loss_bias_start is 1.0
        if loss_bias_start == 1.0 or (is_training_run == False and self.position_loss_bias_in_validation == False):
            seq_mask = ori_seq_mask
        else:
            # Lets get a linear multiplier for the loss bias
            # seq_mask_sum = torch.sum(ori_seq_mask)
            bias_mask = torch.linspace(loss_bias_start, loss_bias_end, int(total_mask_sum.item()), device=ori_seq_mask.device)

            # Boolean flag of seq_mask > 0
            seq_mask_index = ori_seq_mask[0] > 0

            # Apply the bias mask only to positive seq_mask values
            final_mask = torch.zeros(ori_seq_mask.shape[1], device=ori_seq_mask.device)
            final_mask[seq_mask_index] = ori_seq_mask[0][seq_mask_index] * bias_mask

            # And save it as seq_mask
            seq_mask = final_mask.unsqueeze(0)

        # Perform cutoff for training run
        if is_training_run:
            prev_step = 0

            # Avoid using the zip operation, as torch.compile throws an exception on it
            # with `zip not reconized as a valid function`
            # ---
            # for step, len_cut in zip(self.ctx_len_warmup_steps,
            #                          self.ctx_len_cutoffs):
            # ---
            for i in range(min(len(self.ctx_len_warmup_steps), len(self.ctx_len_cutoffs))):
                step = self.ctx_len_warmup_steps[i]
                len_cut = self.ctx_len_cutoffs[i]

                if prev_step <= self.global_step < step and len_cut < seq.shape[
                        1] - 1:
                    pos = randint(0, seq.shape[1] - len_cut - 1)

                    # Original
                    # seq = seq[:, pos:pos + len_cut + 1]

                    # Changed to use masking for prefix cutoff (i do not know if this makes sense)
                    seq = seq[:, :pos + len_cut + 1]
                    seq_mask = seq_mask[:, :pos + len_cut + 1]
                    # Set the attention mask to 0 for the skipped tokens
                    seq_mask[:, :pos] = 0
                    # multitoken mask?
                    print(seq_mask)
                    seq_mask[..., seq_mask == PADDING_IDX] = 0
                    break
                prev_step = step

        do_bptt_learning = self.bptt_learning and is_training_run
        idx, targets = seq[:, :-1], seq[:, 1:]

        B, T, _ = idx.shape
        C = self.n_embd
        total_mask_sum = torch.sum(seq_mask)

        # If total_mask_sum, we skip, as there is no tokens of value to learn from anyway
        if total_mask_sum == 0:
            return 0

        def multi_cross_entropy(input, target, *args, **kwargs):
            target = target.permute(2, 0, 1)
            return sum([F.cross_entropy(input[..., r[1][0]:r[1][1]],
                                        (target[r[0]].view(-1) - r[1][0]) % (r[1][1] - r[1][0]), # TODO how to better prevent overflow due to padding_idx
                                        *args, **kwargs)
                        for r in enumerate(MULTITOKEN_RANGES)])

        def checkpointed_step(idx, targets, mask, prev_loss, last_shift_states,
                              last_wkv_states, prev_steps):
            logits, new_shift_states, new_wkv_states = self(
                idx, last_shift_states, last_wkv_states)

            loss = multi_cross_entropy(logits.view(-1, logits.size(-1)),
                                    targets,
                                    reduction="none")

            submask = mask.view(-1)[:loss.shape[0]]
            submask_sum = torch.sum(submask)
            loss = torch.sum(loss * submask) / total_mask_sum

            loss = L2Wrap.apply(loss, logits, total_mask_sum, submask)
            new_steps = prev_steps + submask_sum
            new_loss = prev_loss + loss
            return new_loss, new_shift_states, new_wkv_states, new_steps

        total_loss = torch.tensor(
            0, dtype=self.emb.weight.dtype).requires_grad_()
        steps = 0
        states = BlockStateList.create(self.n_layer, B, C,
                                       self.n_head, self.head_size,
                                       seq.device, self.emb.weight.dtype)
        segment_count = math.ceil(T / self.ctx_len)

        #
        # BPTT learning, we split the sequence into segments
        # and perform a backward pass for each segment, on its own.
        #
        # Allowing us to perform backpropagation across context sizes much larger
        # then what is supported by the current GPU memory.
        #
        # This reduces the need for the checkpointing process, and mitigate
        # a known error where multiple backwards pass throws an exception.
        #
        # While not mathematically equivalent to full context size learning,
        # it makes "infctx" size training possible with deepspeed 2/3
        #
        # ---
        #
        # See the following, for more details on "Gradient computed twice" error:
        # https://github.com/microsoft/DeepSpeed/issues/988#issuecomment-1549417269
        #
        # Other possibly related issues on the topic:
        # https://github.com/microsoft/DeepSpeed/pull/677
        # https://github.com/EleutherAI/gpt-neox/issues/62#issuecomment-766366413
        #
        if do_bptt_learning:

            gradient_accumulation_steps = max(1, self.trainer.accumulate_grad_batches)
            optimizer = self.optimizers()
            cur_device = self.device

            # We use the average segment size, instead of ctx length size.
            # this helps ensure that the segment cutoffs do not make the last segment too small.
            # (eg, the last chunk having only 1 token)
            #
            # it also helps ensure the segment cutoff points are more varied, across mixed dataset sizes
            # and avoid potentially undesired training behaviour at fixed cutoff points
            # (this only applies for segmented learning)
            segment_size = min(math.ceil(T / segment_count), self.ctx_len)

            # Dummy 2D tenros of shape [1,1], are used to do "dummy checkpoint/forward/backprop" to keep everything in sync
            dummy_2d_zero = torch.tensor([[0]], dtype=torch.long, device=cur_device)

            # Get the max segment count across all GPUs, in the current batch, which is used to keep all devices are in sync
            # Once a thread has completed all its segments, it will do dummy checkpoint/forward/backprop with one token,
            # and stay in sync with the thread that are still working on their segments
            #
            # This is used to work around an undocumented behaviour for either lightning / deepspeed loss.backward multi-gpu handling
            # where the `self.manual_backward()` / `loss.backward()` call will block / freeze / hang when being too "out of sync"
            #
            # This can be viewed as a form of `fabric.barrier()` which is invoked implicitly by each `self.manual_backward()` call
            # except that it isn't exactly a `fabric.barrier()` - because it does not block immediately and instead blocks in
            # the next `self.manual_backward()` call if the previous ones are too far out of sync.
            # (its confusing, but makes sense for efficency)
            #
            # Additionally because the "code line position" and params actually matter for the 'barrier' code effect,
            # we cant work around this issue by doing dummy `self.manual_backward()` calls, in another if/else branch or loop
            #
            # Combined, this makes this issue very hard to trace and debug, as it will manifest itself as randomly "freezing"
            # when out of sync during `self.manual_backward()` calls. Without the current work around put in place
            #
            # We only do this, if we are doing bptt learning on all segments (-1), and gpu count > 1
            # otherwise we just use the segment count as it is
            if self.trainer.num_devices > 1:
                if self.bptt_learning_range <= 0:
                    # We perform forward/backward on the shared max segment count across all GPUs
                    forward_segment_count  = self.trainer.strategy.reduce(segment_count, reduce_op="max")
                    # Convert to int, if its a torch tensor
                    if isinstance(forward_segment_count, torch.Tensor):
                        forward_segment_count = forward_segment_count.item()
                    # We perform as many backward pass as we need to be equal or more then bptt_learning_range
                    backward_segment_count = forward_segment_count
                else:
                    # We perform as many forward pass as we need to be equal or more then bptt_learning_range
                    # and perform an equal amount of backward pass
                    forward_segment_count  = max(segment_count, self.bptt_learning_range)
                    backward_segment_count = self.bptt_learning_range
            else:
                if self.bptt_learning_range <= 0:
                    # Since we do not need to sync GPUs here, we perform as much forward as we exactly need
                    forward_segment_count  = segment_count
                    backward_segment_count = forward_segment_count
                else:
                    # We clamp the backward segment count to the forward count, and bptt_learning_range
                    forward_segment_count  = segment_count
                    backward_segment_count = min(self.bptt_learning_range, segment_count)

            # We compute when we start the segmented learning process
            if forward_segment_count != backward_segment_count:
                start_learning_segment = max(segment_count - self.bptt_learning_range, 0);
            else:
                start_learning_segment = 0;

            # # Segment loss array to track (and reduce later)
            # # of size equal to forward_segment_count
            # segment_loss_arr = [0] * forward_segment_count

            # Lets go through and forward all the segments
            # (including dummy ones)
            for i in range(forward_segment_count):
                # Apply state truncation, if truncated learning is enabled
                # this limits the backprop process, reduces loss learning rate,
                # but save vram across extreamly large backpropagation steps
                if self.bptt_truncated_learning:
                    prv_shift_states = states.shift_states.clone().detach().requires_grad_(False)
                    prv_wkv_states = states.wkv_states.clone().detach().requires_grad_(False)
                else:
                    prv_shift_states = states.shift_states
                    prv_wkv_states = states.wkv_states

                # We use a dummy masked token 0, to do additional dummy checkpoint/forward/backprop when needed
                # for each additional call after the current "segment_count" max
                if i <= segment_count - 1:
                    cur_idx = idx[:, i * segment_size:(i + 1) * segment_size]
                    cur_tar = targets[:, i * segment_size:(i + 1) * segment_size]
                    cur_msk = seq_mask[:, i * segment_size:(i + 1) * segment_size]
                else:
                    cur_idx = dummy_2d_zero
                    cur_tar = dummy_2d_zero
                    cur_msk = dummy_2d_zero

                # Segmented learning, applies the forward/pass over each chunk seperately
                segment_loss, new_shift_states, new_wkv_states, steps = checkpointed_step(
                    cur_idx,
                    cur_tar,
                    cur_msk,
                    torch.tensor(0, dtype=self.emb.weight.dtype, device=cur_device).requires_grad_(True),
                    prv_shift_states,
                    prv_wkv_states,
                    steps,
                )
                states = BlockStateList(new_shift_states, new_wkv_states)

                # # Keep the segment loss (for backpassing in reverse)
                # segment_loss_arr[i] = segment_loss

                # Perform the backward pass accordingly, for valid segments (besides the last segment)
                # In this version, we do backward passes together the forward passes in the main segment loop
                # Instead of after all segment losses are computed
                if i >= start_learning_segment and i < start_learning_segment + backward_segment_count:
                    # The learning loss, should be normalized against the accumulation steps
                    # as we are bypassing the pytorch lightning normalization
                    # https://lightning.ai/docs/pytorch/2.0.4/common/lightning_module.html#backward
                    learning_loss = segment_loss / gradient_accumulation_steps

                    # Perform the backward pass accordingly, for valid segments (besides the last segment)
                    if i == start_learning_segment + backward_segment_count - 1:
                        # This is the last backward pass, we let the default pytorch lightning handle the backward pass
                        # and return the segment loss as part of the total loss
                        total_loss = total_loss + segment_loss
                    else:
                        # Undocumented multiple backward pass support
                        # https://github.com/Lightning-AI/lightning/blob/678f642808c54e4c490caee4df5d357301c976bb/tests/trainer/optimization/test_manual_optimization.py#L251
                        self.manual_backward(learning_loss, optimizer, retain_graph=True)

                        # Accumulate without gradient, as we already did the backward pass
                        total_loss = total_loss + segment_loss.clone().detach().requires_grad_(False)
                else:
                    # Even if its not the segments we use for backward pass, we still need to accumulate the loss
                    total_loss = total_loss + segment_loss.clone().detach().requires_grad_(False)

                # GC collect unused memory
                # gc.collect()
                # torch.cuda.empty_cache()

            # # Lets backpass the respective segments, in reverse
            # # (including dummy backpass)
            # for i in range(forward_segment_count-1, -1, -1):
            #     # Get the segment loss
            #     segment_loss = segment_loss_arr[i]
            #
            #     # Compute the backward pass for the segment
            #     if i >= start_learning_segment and i < start_learning_segment + backward_segment_count:
            #         # The learning loss, should be normalized against the accumulation steps
            #         # as we are bypassing the pytorch lightning normalization
            #         # https://lightning.ai/docs/pytorch/2.0.4/common/lightning_module.html#backward
            #         learning_loss = segment_loss / gradient_accumulation_steps
            #
            #         # Perform the backward pass accordingly, for valid segments (besides the start_learning_segment)
            #         if i > start_learning_segment:
            #             # Undocumented multiple backward pass support
            #             # https://github.com/Lightning-AI/lightning/blob/678f642808c54e4c490caee4df5d357301c976bb/tests/trainer/optimization/test_manual_optimization.py#L251
            #             self.manual_backward(learning_loss, optimizer, retain_graph=True)
            #
            #             # Accumulate without gradient, as we already did the backward pass
            #             total_loss = total_loss + segment_loss.clone().detach().requires_grad_(False)
            #         else:
            #             # This is the last backward pass, we let the default pytorch lightning handle the backward pass
            #             # and return the segment loss as part of the total loss
            #             total_loss = total_loss + segment_loss
            #     else:
            #         # Even if its not the segments we use for backward pass, we still need to accumulate the loss
            #         total_loss = total_loss + segment_loss.clone().detach().requires_grad_(False)
            #
            #    # GC collect unused memory
            #    gc.collect()
            #    # torch.cuda.empty_cache()
        else:

            # Normal operations without BPTT
            segment_size = self.ctx_len
            for i in range(segment_count):
                if i < segment_count-1 and is_training_run:
                    total_loss, new_shift_states, new_wkv_states, steps = deepspeed_checkpoint(
                        checkpointed_step,
                        idx[:, i * segment_size:(i + 1) * segment_size],
                        targets[:, i * segment_size:(i + 1) * segment_size],
                        seq_mask[:, i * segment_size:(i + 1) * segment_size],
                        total_loss,
                        states.shift_states,
                        states.wkv_states,
                        steps,
                    )
                else:
                    total_loss, new_shift_states, new_wkv_states, steps = checkpointed_step(
                        idx[:, i * segment_size:(i + 1) * segment_size],
                        targets[:, i * segment_size:(i + 1) * segment_size],
                        seq_mask[:, i * segment_size:(i + 1) * segment_size],
                        total_loss,
                        states.shift_states,
                        states.wkv_states,
                        steps,
                    )

                states = BlockStateList(new_shift_states, new_wkv_states)
                gc.collect()
                # torch.cuda.empty_cache()

        # Wandb logging only, if an active run exists
        if wandb.run is not None:
            global_rank = self.global_rank
            global_device_count = self.trainer.num_devices * self.trainer.num_nodes
            wandb.log({
                'substep': batch_idx * global_device_count + global_rank,
                'batchidx': batch_idx,
                'global_rank': global_rank,
                'real_ctx_len': T,
                'train/loss': total_loss,
                'trainer/global_step':self.global_step,
                'trainer/learning_rate': self.trainer.optimizers[0].param_groups[0]['lr']
            })

        return total_loss

    @TCompileBaseline
    def training_step(self, batch, batch_idx):
        total_loss = self.compute_loss(batch, batch_idx, True)

        self.log('train/loss', total_loss, prog_bar=True)
        # If set - forces the above train/loss log line to always be on a new line
        if self.substep_logging:
            print("")

        if self.substep_cuda_cache_clear:
            gc.collect()
            torch.cuda.empty_cache()

        return total_loss

    @TCompileBaseline
    def validation_step(self, batch, batch_idx):
        total_loss = self.compute_loss(batch, batch_idx, False)
        self.log('validation/loss', total_loss, prog_bar=True, sync_dist=True)
        return total_loss

########################################################################################################
# SimpleRWKV, a wrapper for RWKV that allows for simple usage of the model
########################################################################################################

# SimpleRWKV specific imports
from transformers import PreTrainedTokenizerFast

# Current script dir
SCRIPT_DIR = os.path.dirname(os.path.realpath(__file__))
SCRIPT_PARENT_DIR = os.path.abspath(os.path.join(SCRIPT_DIR, '../'))

# SimpleRWKV is a wrapper for RWKV that allows for simple usage of the model
#
# it is not meant to be highly performant, but rather a simple minimal way to run the RWKV trainer module
# in inference mode, and can be used to validate the model trainer code / its changes
class SimpleRWKV():

    def __init__(
            self,
            model_path: str,
            ctx_len:int = 1024,
            device:str = "cuda",
            dtype:str = "fp32"
        ):

        # Log the mismatch dtype
        if dtype != "fp32":
            print("[SimpleRWKV] Warning: dtype mismatch, only fp32 is supported (for now)")

        # Prepare the model config with the model path, and custom torch load
        model_config = {}
        model_config["load_model"] = model_path
        model_config["ctx_len"] = ctx_len

        # This feature depends on deepspeed
        model_config["grad_cp"] = False
        # model_config["_torch_load_state"] = loaded_state

        # Save the config settings
        self.ctx_len = ctx_len
        self.device = device

        # Lets actually load the model
        self.model = RWKV(**model_config)

        # Lets map it over to the respective device type
        # and set it to run as eval/inference mode
        self.model.to(device)
        self.model.eval()

        # Get the model detected vocab size
        vocab_size = self.model.vocab_size

        # The tokenizer object values
        self.fastTokenizer = None
        self.worldTokenizer = None

        # Setup the tokenizer
        if vocab_size == 50277:
            # Use the neox tokenizer
            tokenizer_file = os.path.join(SCRIPT_DIR,"./dataflow/20B_tokenizer.json")
            tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file)
            self.fastTokenizer = tokenizer
        elif vocab_size == 65536:
            # Use the world tokenizer
            from .dataflow.trie_tokenizer import MT_TRIE_TOKENIZER
            world_tokenizer = MT_TRIE_TOKENIZER(os.path.join(SCRIPT_DIR, "./dataflow/rwkv_vocab_v20230424.txt"))
            self.worldTokenizer = world_tokenizer
        else:
            raise NotImplementedError(f"Unsupported vocab size ({vocab_size}) - custom tokenizer not supported")

    # Encoding strings
    def encode(self, text: str):
        if self.worldTokenizer != None:
            return self.worldTokenizer.encode(text)
        return self.fastTokenizer.encode(text)

    # Decoding strings
    def decode(self, tokens: list):
        if self.worldTokenizer != None:
            return self.worldTokenizer.decode(tokens)
        return self.fastTokenizer.decode(tokens)

    # Forwarding logic, withoout torch._no_grad() context
    def _forward(
            self, tokens,
            stateObj = None,
            all_logits = False
        ):

        logits_arr = None
        token_len = len(tokens)

        # Get the shift/wkv state
        if stateObj is None:
            shift_states = None
            wkv_states = None
        else:
            shift_states = stateObj["shift_states"]
            wkv_states = stateObj["wkv_states"]

        # The all_logits array, if requested
        all_logits_arr = None

        # For each token, process the state, in batches up to ctx_len
        for i in range(0, token_len, self.ctx_len):
            # Token set
            token_set = tokens[i:i+self.ctx_len]

            # Check if tokens are already tensors
            batch_tokens = torch.tensor(
                token_set,
                dtype=torch.long, device=self.device
            ).unsqueeze(0)

            # Compute the logits and state
            logits_arr, shift_states, wkv_states = self.model.forward(
                batch_tokens, shift_states, wkv_states
            )

            # Build the all_logits array
            if all_logits:
                if all_logits_arr is None:
                    all_logits_arr = logits_arr[0]
                else:
                    all_logits_arr = torch.cat([all_logits_arr, logits_arr[0]], dim=0)

        # Return the logits and state
        if all_logits:
            return all_logits_arr, { "shift_states": shift_states, "wkv_states": wkv_states }
        else:
            return logits_arr[0][-1], { "shift_states": shift_states, "wkv_states": wkv_states }

    # Forwarding logic, with torch._no_grad() context
    def forward(
            self, tokens:list,
            stateObj = None,
            all_logits = False
        ):
        with torch.no_grad():
            return self._forward(tokens, stateObj, all_logits)

    # Sampling logits
    def sample_logits(
            self, logits,
            prv_tokens=[0],
            temperature=1.0, top_p=0.9,
            token_ban: list = []
            ):
        # Copy to CPU first
        logits = logits.cpu()

        # Max negative float
        max_neg = -torch.finfo(torch.float).max

        # Apply token ban
        for x in token_ban:
            logits[x] = max_neg

        # Remove NaNs from logits
        for x in range(len(logits)):
            if torch.isnan(logits[x]):
                logits[x] = max_neg

        # Handle sampling with temperature
        if temperature > 0.0:
            probs = F.softmax(logits, dim=-1)
            sorted_probs = torch.sort(probs, descending=True)[0]
            cumulative_probs = torch.cumsum(sorted_probs, dim=-1).cpu().numpy()
            cutoff = float(sorted_probs[np.argmax(cumulative_probs > top_p)])
            probs[probs < cutoff] = 0
            if temperature != 1.0:
                probs = probs.pow(1.0 / temperature)
            out = torch.multinomial(probs, num_samples=1)[0]
            return out
        else:
            # Since the tokenizer sample does not support temp==0
            # we handle this case ourself, by fining the top token
            return torch.argmax(logits, dim=-1).item()

    # Completion API
    def completion(self,
            prompt,
            max_tokens: int = 32,
            temperature: float = 1.0,
            top_p: float = 0.9,
            token_ban: list = [],
            start_state = None,
            stream_to_stdout: bool = False,
        ):
        # Encode the context, if its a string
        if isinstance(prompt, str):
            enc = self.encode(prompt)
        # Check if the prompt is a list of tokens
        elif isinstance(prompt, list):
            enc = prompt
        else:
            raise ValueError("Prompt must be a string or a list of tokens")

        # Keep track of the logits and state
        logits = None
        stateObj = start_state

        # For each token, process the state
        logits, stateObj = self.forward(enc, stateObj)

        # # Garbage collect
        # gc.collect()
        # torch.cuda.empty_cache()

        # Generate each token
        out_tokens = []
        for i in range(max_tokens):
            ttt = self.sample_logits(
                logits,
                # prv_tokens=full_tokens,
                temperature=temperature, top_p=top_p,
                token_ban=token_ban
            )

            # Append the token
            out_tokens.append(ttt)
            # full_tokens.append(ttt)
            if stream_to_stdout:
                print(self.decode([ttt]), end="", flush=True)

            # Perform the forward pass
            logits, stateObj = self.forward([ttt], stateObj)

        # Decode the tokens
        out_str = self.decode(out_tokens)

        # # Garbage collect
        # gc.collect()
        # torch.cuda.empty_cache()

        # Return the output string, and state
        return out_str, stateObj


Overwriting /content/RWKV-infctx-trainer/RWKV-v5/src/model.py


In [ ]:
#@title patch trainer.py

%%writefile /content/RWKV-infctx-trainer/RWKV-v5/src/trainer.py

from lightning.pytorch import Trainer
from lightning.pytorch.strategies.deepspeed import DeepSpeedStrategy
import lightning as Lightning
import torch
import math
import wandb

# We extend the native pytorch lightning trainer to add the following
#
# - local "fabric" support, as the trainer object is one of the few
#   objects that is available to all the processess
# - target_batch_size, which automatically computes the accumulate_grad_batches
class RWKVLightningTrainer(Trainer):
    def __init__(
            self,
            *args,
            # Replaces the accumulate_grad_batches, if set
            # automatically compute the accumulate_grad_batches
            # according to the num_nodes, and num_devices configured
            target_batch_size=-1,
            # Handle the rest of args, as per normal
            **kwargs,
        ):

        # trainer_config args (used for wndb logging)
        trainer_config = dict(kwargs)

        # target batch size logging
        target_batch_size_log_msg = ""

        # Compute the accumulate_grad_batches, using the target_batch_size
        self.target_batch_size = target_batch_size
        if target_batch_size > 0:

            # Check if the accumulate_grad_batches is already set
            # (note that it seems that pytorch lightning defaults to 1)
            if "accumulate_grad_batches" in kwargs and kwargs["accumulate_grad_batches"] > 1:
                raise ValueError(f"Cannot set both 'target_batch_size' ({target_batch_size}) and 'accumulate_grad_batches' ({kwargs['accumulate_grad_batches']}))")

            # Extract the num_nodes and devices
            num_nodes = kwargs.get("num_nodes", 1)
            devices = kwargs.get("devices", "auto")

            # Compute the number of devices
            if devices == "auto":
                num_devices = torch.cuda.device_count()
            elif isinstance(devices, int):
                num_devices = devices
            elif isinstance(devices, list):
                num_devices = len(devices)
            else:
                raise ValueError(f"Unsupported devices config '{devices}', unable to compute device count for 'target_batch_size'")

            # Compute the accumulate_grad_batches
            accumulate_grad_batches = max( 1, math.floor(target_batch_size / (num_nodes * num_devices)) )
            kwargs["accumulate_grad_batches"] = accumulate_grad_batches
            effective_batch_size = accumulate_grad_batches * num_nodes * num_devices

            # Log the applied accumulate_grad_batches
            trainer_config["__accumulate_grad_batches"] = accumulate_grad_batches
            trainer_config["__effective_batch_size"] = effective_batch_size

            # Log the computed accumulate_grad_batches
            # this is done after _init_ so we can confirm local rank
            target_batch_size_log_msg = ("\n"+
                f"\n[RWKV.Trainer] Applying 'target_batch_size' with the following:\n"+
                f"   - target_batch_size:       {target_batch_size}\n"+
                f"   - num_nodes:               {num_nodes}\n"+
                f"   - num_devices:             {num_devices}\n"+
                f"   - accumulate_grad_batches: {accumulate_grad_batches}\n"
                f"   - effective_batch_size:    {effective_batch_size}\n")

        # Update WANDB config
        # ---
        if wandb.run is not None:
            trainer_config["target_batch_size"] = target_batch_size
            del trainer_config["logger"]
            del trainer_config["callbacks"]
            wandb.config.update({
                "trainer": trainer_config
            })

        # Call the parent constructor
        super().__init__(*args, **kwargs)
        self._fabric_instance = None

        # Log the target_batch_size_log_msg
        # if local rank is 0
        if target_batch_size_log_msg != "" and self.local_rank == 0:
            print(target_batch_size_log_msg)

    # Fabric instance, useful for coordinating between processes
    # when `self.trainer.strategy.reduce` is not possible
    def getFabric(self):
        if self._fabric_instance is not None:
            return self._fabric_instance

        strat = self.strategy
        if strat is None:
            raise ValueError("Trainer strategy config is missing")

        # Map the pytorch lightning strat to fabric strat string
        stratStr = "auto"
        if isinstance(strat, DeepSpeedStrategy):
            stratStr = "deepspeed"

        self._fabric_instance = Lightning.Fabric(
            accelerator=self.accelerator,
            devices=self.num_devices,
            num_nodes=self.num_nodes,
            strategy=stratStr
        )
        return self._fabric_instance


Overwriting /content/RWKV-infctx-trainer/RWKV-v5/src/trainer.py


In [ ]:
!pip install --upgrade protobuf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
descript-audiotools 0.7.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.24.2 which is incompatible.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.24.2 which is incompatible.


In [ ]:
!python /content/RWKV-infctx-trainer/RWKV-v5/init_model.py --n_layer 12 --n_embd 1024 --vocab_size 65536 --skip-if-exists /content/init.pth

2023-09-06 11:57:33.066302: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2023-09-06 11:57:35,439] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
---- Initializing model ----
No of layers: 12
Embedding size: 1024
Output model path: /content/init.pth
Vocab size: 65536
Emb scale: 0.0001
Note: this process takes a significant time (and ram) for large models
---- ----- ----
65536 1024  -0.0001 emb.weight
1024  1024  1.0  blocks.0.att.receptance.weight
1024  1024  1.0  blocks.0.att.key.weight
1024  1024  1.0  blocks.0.att.value.weight
1024  1024  0    blocks.0.att.output.weight
4096  1024  1.0  blocks.0.ffn.key.weight
1024  1024  0    blocks.0.ffn.receptance.weight
1024  4096  0    blocks.0.ffn.value.weight
1024  1024  1.0  blocks.1.att.receptance.weight
1024  1024  1.0  blocks.1.att.key.weight
1024  1024  1.0  blocks.1.

In [ ]:
!rm -r /content/RWKV-infctx-trainer/RWKV-v5/HF

rm: cannot remove '/content/RWKV-infctx-trainer/RWKV-v5/HF': No such file or directory


In [ ]:
%cd /content/RWKV-infctx-trainer/RWKV-v5
!python lightning_trainer.py fit -c /content/config.yaml --seed 150

/content/RWKV-infctx-trainer/RWKV-v5
2023-09-06 11:58:54.179300: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2023-09-06 11:58:56,375] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/content/config.yaml', '--seed', '150'], args=['fit', '-c', '/content/config.yaml', '--seed', '150'].
  rank_zero_warn(
Global seed set to 150
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 2
wandb: You chose 'Use an existing W&B account'
wan